In [7]:
use_provider = 1
missing_val_cutoff = .5
drop_outlier_homes = 1
use_clusters = 1
num_weeks_staggered = 3

https://data.cms.gov/covid-19/covid-19-nursing-home-data

https://data.cms.gov/provider-data/dataset/4pq5-n9py

In [8]:
path_to_covid_data = '/content/drive/MyDrive/COVID_19_Nursing_Home_Data_03_06_2022.csv'
path_to_provider_data = '/content/drive/MyDrive/NH_ProviderInfo_Feb2022.csv'

#Read in Data

In [9]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [10]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm

pd.set_option('display.max_columns', None)

In [11]:
df = pd.read_csv(path_to_covid_data)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
df['Date'] = pd.to_datetime(df['Week Ending'])

# Convert to weeks since the start of the data
weeks = df['Week Ending'].unique()
mapping = dict(zip(weeks, range(0, len(weeks))))
df['Week Number'] = df['Week Ending'].replace(mapping)
df = df.drop('Week Ending', axis=1)

In [13]:
# 846/15000 homes are not in this data, can just do an inner join
if use_provider:
  prov_df = pd.read_csv(path_to_provider_data)
  df['Federal Provider Number'] = df['Federal Provider Number'].astype(str)
  prov_df['Federal Provider Number'] = prov_df['Federal Provider Number'].astype(str)
  prov_df.drop([
       'Provider Name', 
       'Provider Address',
       'Provider City', 
       'Provider State', 
       'Provider Zip Code',
       'Provider Phone Number', 
       'Provider SSA County Code',
       'Provider County Name',
       'Legal Business Name',
       'Date First Approved to Provide Medicare and Medicaid Services',
       'Rating Cycle 2 Standard Health Survey Date',
       'Location',
       'Processing Date',
       'Rating Cycle 1 Standard Survey Health Date',
       'Rating Cycle 1 Total Number of Health Deficiencies',
       'Rating Cycle 1 Number of Standard Health Deficiencies',
       'Rating Cycle 1 Number of Complaint Health Deficiencies',
       'Rating Cycle 1 Health Deficiency Score',
       'Rating Cycle 1 Number of Health Revisits',
       'Rating Cycle 1 Health Revisit Score',
       'Rating Cycle 1 Total Health Score',
       'Rating Cycle 2 Total Number of Health Deficiencies',
       'Rating Cycle 2 Number of Standard Health Deficiencies',
       'Rating Cycle 2 Number of Complaint Health Deficiencies',
       'Rating Cycle 2 Health Deficiency Score',
       'Rating Cycle 2 Number of Health Revisits',
       'Rating Cycle 2 Health Revisit Score',
       'Rating Cycle 2 Total Health Score',
       'Rating Cycle 3 Standard Health Survey Date',
       'Rating Cycle 3 Total Number of Health Deficiencies',
       'Rating Cycle 3 Number of Standard Health Deficiencies',
       'Rating Cycle 3 Number of Complaint Health Deficiencies',
       'Rating Cycle 3 Health Deficiency Score',
       'Rating Cycle 3 Number of Health Revisits',
       'Rating Cycle 3 Health Revisit Score',
       'Rating Cycle 3 Total Health Score'], inplace=True, axis=1)
  df = df.merge(prov_df, left_on=['Federal Provider Number'], right_on=['Federal Provider Number'], how='inner')

#Data Cleaning

In [14]:
df = df.sort_values(['Federal Provider Number', 'Week Number'])

In [15]:
#Drop all columns with >x% missing values
for col in df.columns:
  percent_missing = df[col].isna().sum()/(df.shape[0] * 1.0)
  if percent_missing > missing_val_cutoff:
    df = df.drop(col, axis=1)

In [16]:
#Drop all rows with missing y values
df = df.dropna(subset=['Residents Weekly Confirmed COVID-19'])

In [17]:
# Drop rows that did not submit data or didn't pass the quality check
df = df[df['Passed Quality Assurance Check'] == 'Y']
df = df[df['Submitted Data'] == 'Y']

In [18]:
# Drop obviously useless columns

df = df.drop(['Provider Name', 
              'Provider Address', 
              'Provider City', 
              'Provider Phone Number', 
              'Submitted Data',
              'Passed Quality Assurance Check',
              'Reporting Interval',
              'Provider State',
              'Provider Zip Code'], axis=1)

In [19]:
#Drop homes that never had a covid case, or extreme outliers
if drop_outlier_homes:
  g_df = df.groupby('Federal Provider Number').mean()
  provider_numbers = list(g_df[g_df['Residents Weekly Confirmed COVID-19'] == 0].index)
  provider_numbers += list(g_df[g_df['Residents Weekly Confirmed COVID-19'] > 4].index)
  df = df[~df['Federal Provider Number'].isin(provider_numbers)]
  df.shape

In [20]:
# Interpolate data
df[df.columns.drop('Date')] = df[df.columns.drop('Date')].interpolate(method='nearest') 

df = df.fillna(0)

#Data Encoding

In [21]:
# One hot encode categorical columns
cat_cols = df.select_dtypes(include=['object']).columns.drop(['Federal Provider Number', 'County'])

one_hot_cols = []

for col in cat_cols:
  one_hot = pd.get_dummies(df[col], prefix=col, drop_first=True)
  one_hot_cols += list(one_hot.columns)
  df = df.drop(col, axis=1)
  df = df.join(one_hot)

In [22]:
from sklearn.cluster import KMeans

# Cluster homes
if use_clusters:
  g_df = df.groupby('Federal Provider Number').mean()
  model = KMeans(n_clusters = 50, random_state=42).fit(g_df)
  g_df['Cluster'] = model.labels_
  g_df['Federal Provider Number'] = g_df.index

  df = df.merge(g_df['Cluster'], left_on='Federal Provider Number', right_on='Federal Provider Number')

  one_hot = pd.get_dummies(df['Cluster'], prefix='Cluster')
  cluster_cols = list(one_hot.columns)
  one_hot_cols += cluster_cols
  df = df.drop('Cluster', axis=1)
  df = df.join(one_hot)

In [23]:
# Aggregate our data into counties
df = df.groupby(['Date', 'Week Number', 'County']).sum().reset_index()

Stagger data so we're predicting this weeks cases on the last X weeks data

In [24]:
df = df.sort_values(['Week Number'])

cols_not_staggered = ['Week Number', 'Date'] 

# Don't stagger cluster labels
if use_clusters:     
  cols_not_staggered = cols_not_staggered + cluster_cols

# Don't stagger provider columns
if use_provider:
  prov_cols = [x for x in prov_df.columns if x in df.columns]
  cols_not_staggered = cols_not_staggered + prov_cols

cols_to_stagger = df.columns.drop(cols_not_staggered + ['County'])

staggered_df = pd.DataFrame()

for county in tqdm(df['County'].unique()):
  county_df = df.loc[df['County'] == county].reset_index(drop=True) #all data for homes in this county
  for i in range(1, num_weeks_staggered + 1):
    staggered_county = county_df[cols_to_stagger].shift(i)
    county_df = county_df.join(staggered_county, rsuffix=('_' + str(i) + '_weeks_ago'))
  staggered_df = staggered_df.append(county_df)

#Shift creates nans if there aren't enough values to stagger, so just drop those weeks
staggered_df = staggered_df.dropna()

staggered_df = staggered_df.sort_values('Week Number').reset_index(drop=True)

100%|██████████| 1657/1657 [03:28<00:00,  7.93it/s]


In [25]:
X = staggered_df.drop(list(cols_to_stagger) + ['County', 'Date'], axis=1)
y = staggered_df['Residents Weekly Confirmed COVID-19']

In [26]:
#Standardize predictors
from sklearn.preprocessing import StandardScaler

# Ignore binary columns
cont_cols = [c for c in X.columns if len(X[c].unique()) > 2]

scaler = StandardScaler()
X[cont_cols] = scaler.fit_transform(X[cont_cols])

#Data Modeling

In [27]:
from sklearn.linear_model import Ridge, LinearRegression, ElasticNet, Lasso, LassoLars
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import BaggingRegressor, RandomForestRegressor
from sklearn.neighbors import KNeighborsRegressor
from sklearn.svm import SVR
from sklearn.model_selection import TimeSeriesSplit
from sklearn.metrics import mean_squared_error, r2_score

def runSKModelsReg(X_train, X_test, y_train, y_test):
  res = []
  for model in [Ridge(),
                ElasticNet(),
                Lasso(),
                DecisionTreeRegressor(),
                BaggingRegressor(Ridge(), max_samples=0.5, max_features=0.5)
                ]:
    model.fit(X_train, y_train)
    pred = model.predict(X_test)
    error = mean_squared_error(y_test, pred, squared=False)
    print(round(error, 3), round(r2_score(y_test, pred), 3),  model)
    res.append(error)
    del(model)
  return res

In [28]:
import tensorflow as tf

def runMLPReg(X_train, X_test, y_train, y_test):
  model = tf.keras.models.Sequential([
        tf.keras.layers.Dense(128,activation=tf.nn.relu,input_shape=[X_train.shape[1],]),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(64,activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(64,activation=tf.nn.relu),
        tf.keras.layers.Dropout(0.1),
        tf.keras.layers.Dense(64,activation=tf.nn.relu),
        tf.keras.layers.Dense(32,activation=tf.nn.relu),
        tf.keras.layers.Dense(1)
        ])

  model.compile(optimizer='adam', loss='mean_squared_error')

  history = model.fit(X_train,y_train,epochs=5,validation_split=0.2,batch_size=100, verbose=False)
  pred =  model.predict(X_test)
  error = mean_squared_error(y_test, pred, squared=False)
  print(round(error, 3), round(r2_score(y_test, pred), 3), 'Fully Connected Neural Network')

  return error

def runRNNReg(X_train, X_test, y_train, y_test):
  X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
  X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
  y_train = y_train.reshape(-1, 1)
  y_test = y_test.reshape(-1, 1)

  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.SimpleRNN(128, input_shape=X_train[0].shape, return_sequences=True))
  model.add(tf.keras.layers.Dropout(0.1))
  model.add(tf.keras.layers.SimpleRNN(64, activation=tf.nn.relu, return_sequences=True))
  model.add(tf.keras.layers.Dropout(0.1))
  model.add(tf.keras.layers.SimpleRNN(64, activation=tf.nn.relu))
  model.add(tf.keras.layers.Dropout(0.1))
  model.add(tf.keras.layers.Dense(32, activation=tf.nn.relu))
  model.add(tf.keras.layers.Dense(1))

  model.compile(optimizer='adam', loss='mean_squared_error')

  history = model.fit(X_train,y_train,epochs=2,validation_split=0.2,batch_size=100, verbose=False)

  pred = model.predict(X_test).reshape(-1)
  y_test = y_test.reshape(-1)
  error = mean_squared_error(y_test, pred, squared=False)
  print(round(error, 3), round(r2_score(y_test, pred), 3), 'Recurrent Neural Network')

  return error

def runLSTMReg(X_train, X_test, y_train, y_test):
  X_train = X_train.reshape(X_train.shape[0], 1, X_train.shape[1])
  X_test = X_test.reshape(X_test.shape[0], 1, X_test.shape[1])
  y_train = y_train.reshape(-1, 1)
  y_test = y_test.reshape(-1, 1)

  model = tf.keras.models.Sequential()
  model.add(tf.keras.layers.LSTM(64, input_shape=X_train[0].shape, return_sequences=True))
  model.add(tf.keras.layers.LSTM(32,activation=tf.nn.relu, return_sequences=True))
  model.add(tf.keras.layers.LSTM(32,activation=tf.nn.relu, return_sequences=True))
  model.add(tf.keras.layers.Dense(16,activation=tf.nn.relu))
  model.add(tf.keras.layers.Dense(1))

  model.compile(optimizer='adam', loss='mean_squared_error')

  history = model.fit(X_train,y_train,epochs=2,validation_split=0.2,batch_size=100,verbose=False)

  pred = model.predict(X_test).reshape(-1)
  y_test = y_test.reshape(-1)
  error = mean_squared_error(y_test, pred, squared=False)
  print(round(error, 3), round(r2_score(y_test, pred), 3), 'LSTM Neural Network')

  return error

In [29]:
y = np.array(y)

tscv = TimeSeriesSplit()
num_split = 0
all_results = np.ndarray((5, 8))
for train_index, test_index in tscv.split(X):
  X_train, X_test = X.values[train_index], X.values[test_index]
  y_train, y_test = y[train_index], y[test_index]

  print(num_split, '----------------------------------------------------------')

  results = runSKModelsReg(X_train, X_test, y_train, y_test)
  results.append(runMLPReg(X_train, X_test, y_train, y_test))
  results.append(runRNNReg(X_train, X_test, y_train, y_test))
  results.append(runLSTMReg(X_train, X_test, y_train, y_test))
  
  all_results[num_split] = np.array(results)
  num_split += 1

all_results = np.round(np.mean(all_results, axis=0), decimals=3)
print(all_results, all_results.min())
print("use_provider:", use_provider, ", missing_val_cutoff:", missing_val_cutoff, ", drop_outlier_homes:", drop_outlier_homes, ", use_clusters:", use_clusters, ", num_weeks_staggered:", num_weeks_staggered)

0 ----------------------------------------------------------
16.878 0.763 Ridge()
22.287 0.586 ElasticNet()
20.632 0.646 Lasso()
23.276 0.549 DecisionTreeRegressor()
17.318 0.75 BaggingRegressor(base_estimator=Ridge(), max_features=0.5, max_samples=0.5)
25.326 0.466 Fully Connected Neural Network
24.741 0.49 Recurrent Neural Network
25.466 0.46 LSTM Neural Network
1 ----------------------------------------------------------
8.901 0.657 Ridge()
9.174 0.635 ElasticNet()
7.712 0.742 Lasso()
18.029 -0.408 DecisionTreeRegressor()
9.155 0.637 BaggingRegressor(base_estimator=Ridge(), max_features=0.5, max_samples=0.5)
8.652 0.676 Fully Connected Neural Network
125.063 -66.744 Recurrent Neural Network
27.367 -2.244 LSTM Neural Network
2 ----------------------------------------------------------
3.943 -1.296 Ridge()
1.911 0.461 ElasticNet()
1.828 0.506 Lasso()
2.557 0.035 DecisionTreeRegressor()
3.073 -0.394 BaggingRegressor(base_estimator=Ridge(), max_features=0.5, max_samples=0.5)
5.049 -2.76

In [30]:
# What features are the most important?
model = Lasso().fit(X_train, y_train)
pred =  model.predict(X_test)
print(round(mean_squared_error(y_test, pred, squared=False), 3), round(r2_score(y_test, pred), 3))

temp = pd.DataFrame(model.coef_, columns=['Coefficient'])
temp['Column'] = X.columns
temp = temp.sort_values('Coefficient')
temp

15.927 0.841


,Coefficient,Column
0,-0.000000,Week Number
316,0.000000,Ownership Type_For profit - Limited Liability ...
315,0.000000,Ownership Type_For profit - Individual_2_weeks...
314,-0.000000,Facility Would Like Assistance Outreach for Va...
313,0.000000,Facility Would Like Assistance Outreach for Va...
...,...,...
140,1.127473,Initial Confirmed COVID-19 Case This Week_Y_1_...
110,1.463577,Number of Residents with a New Positive COVID-...
138,3.561761,Three or More Confirmed COVID-19 Cases This We...
99,6.704452,Staff Weekly Confirmed COVID-19_1_weeks_ago


#Export Predictions

In [31]:
model = Lasso().fit(X_train, y_train)
pred =  model.predict(X_test)

In [32]:
pred_df = pd.DataFrame(data=pred.reshape(-1, 1), columns=['Prediction'])
pred_df['Actual Cases'] = y_test
pred_df['County'] = staggered_df['County'].iloc[test_index].reset_index(drop=True)
pred_df['Date'] = staggered_df['Date'].iloc[test_index].reset_index(drop=True)

In [36]:
#Make a prediction one week into the future
this_week_df = staggered_df[staggered_df['Week Number'] == max(staggered_df['Week Number'])]

this_week_counties = this_week_df['County']

this_week_df = this_week_df.drop(['County', 'Date'], axis=1)

column_name_mapper = {}

for c in this_week_df.columns:
  if c.endswith('_3_weeks_ago'):
    this_week_df = this_week_df.drop(c, axis=1)
  elif c.endswith('_2_weeks_ago'):
    column_name_mapper[c] = c.replace('_2_weeks_ago', '_3_weeks_ago')
  elif c.endswith('_1_weeks_ago'):
    column_name_mapper[c] = c.replace('_1_weeks_ago', '_2_weeks_ago')
  elif c in cols_to_stagger:
    column_name_mapper[c] = c + '_1_weeks_ago'

this_week_df = this_week_df.rename(mapper=column_name_mapper, axis=1)

this_week_df = this_week_df[X.columns.tolist()] # Reorder to X's column order

this_week_df[cont_cols] = scaler.transform(this_week_df[cont_cols])

In [37]:
import datetime

next_week_df = pd.DataFrame()
next_week_df['Prediction'] = Lasso().fit(X, y).predict(this_week_df)
next_week_df['Actual Cases'] = np.zeros(len(next_week_df))
next_week_df['County'] = this_week_counties.values
next_week_df['Date'] = [max(staggered_df['Date']) + datetime.timedelta(days=7)] * len(next_week_df)

In [38]:
pred_df = pd.concat([pred_df, next_week_df])
pred_df = pred_df.sort_values('Date')

In [39]:
result_df = pd.DataFrame()
for county in pred_df['County'].unique():
  county_df = pred_df.loc[pred_df['County'] == county].reset_index(drop=True) #all data for homes in this county
  staggered_county = county_df[['Actual Cases', 'Prediction']].shift(1)
  county_df = county_df.join(staggered_county, rsuffix=(' Lag'))
  result_df = result_df.append(county_df)

result_df = result_df.fillna(0)
result_df['Weekly Predicted Difference'] = result_df['Prediction'] - result_df['Actual Cases Lag']

In [40]:
result_df.to_csv('County Level Predictions.csv')